# Traitement des données de financements 

In [1]:
#importer les packages
import pandas as pd
import numpy as np
from project.server.main.utils import replace_all,get_scanR_structure,get_id
from project.server.main.cached_data_handler import get_structure, get_person
from project.server.main.features_into_dictionnary import persons, projects, address
from project.server.main.id_from_orcid import orcid_to_idref
from tqdm import tqdm
import pprint as pp
tqdm.pandas()
from project.server.main.my_pickle import load_cache,write_cache
import os
from dotenv import load_dotenv
import requests

load_dotenv()

Authorization = os.getenv('Authorization_access_185.XX')
Authorization_ORCID = os.getenv('Authorization_cluster_BSO_ORCID')
url_cluster = os.getenv('url_cluster')

In [2]:
sources=pd.read_json('sources.json')

In [3]:
source=list(sources.keys())[1]
source

'ANSES'

In [4]:
#cache structures, personnes et orcid avec differentes sources de donnees
cached_data = {}
try:
    cached_data = load_cache(cached_data,f"./DATA/{source}/caches/cached_{source.lower()}_data.pkl")
except:
    write_cache(cached_data,f"./DATA/{source}/caches/cached_{source.lower()}_data.pkl")
    
cached_data_persons = {}
try:
    cached_data_persons = load_cache(cached_data_persons,f"./DATA/{source}/caches/cached_{source.lower()}_data_persons.pkl")
except:
    write_cache(cached_data_persons,f"./DATA/{source}/caches/cached_{source.lower()}_data_persons.pkl")
    
cached_data_orcid = {}
try:
    cached_data_orcid = load_cache(cached_data_orcid,f"./DATA/{source}/caches/cached_{source.lower()}_data_orcid.pkl")
except:
    write_cache(cached_data_orcid,f"./DATA/{source}/caches/cached_{source.lower()}_data_orcid.pkl")

969 data in cache
0 data in cache
0 data in cache


# Données partenaires

In [5]:
# amener les partenaires depuis le site 
if source=='ANR':
    page_partenaires_10 = requests.get(sources[source]['url_partners']).json()
    colonnes_partenaires_10 = page_partenaires_10['columns']
    donnees_partenaires_10 = page_partenaires_10['data']
    df_partenaires=pd.DataFrame(data=donnees_partenaires_10,columns=colonnes_partenaires_10)
elif source=='ANSES':
    df_from_anses=pd.read_excel(sources[source]['url_partners'])
    df=df_from_anses.iloc[1:,:]
    df.columns=list(df_from_anses.iloc[0,:])
    dict_equipe={list(df_from_anses.columns)[k].replace('Équipe 10 ','Équipe 10').replace('Équipe13','Équipe 13'):k for k in range (len(list(df_from_anses.columns))) if list(df_from_anses.columns)[k].find('Équipe')>=0}
    list_df=[]
    number=3
    for n in range(1,len(dict_equipe)+1):
        equipe_n=pd.concat([df.iloc[:,0:3],df.iloc[:,number:number+6]], axis=1)
        list_df.append(equipe_n)
        number+=6
    df_partenaires=pd.concat([list_df[k].dropna(subset=[sources[source]['nom'], sources[source]['prenom'],sources[source]['nom_structure'], sources[source]['nom'], 'Pays'], how='all') for k in range(len(list_df))])
    
elif source=='IRESP':
    df_partenaires1=pd.read_csv(sources[source]['url_partners1'] ,sep=";", encoding='UTF-8')
    df_partenaires2=pd.read_csv(sources[source]['url_partners2'] ,sep=";", encoding='UTF-8')
    df_partenaires=pd.concat([df_partenaires1,df_partenaires2])
elif source=='ADEME':
    df_partenaires=pd.read_csv(sources[source]['url_partners'] ,sep=",", encoding='ISO-8859-1', on_bad_lines='skip')
else:    
    df_partenaires=pd.read_csv(sources[source]['url_partners'] ,sep=";", encoding='ISO-8859-1')

df_partenaires=df_partenaires.reset_index()
del df_partenaires['index']

# Matcher établissement

In [6]:
id_struct=df_partenaires
id_struct[f"{sources[source]['nom_structure']}2"]=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
id_struct=id_struct.drop_duplicates(subset=[f"{sources[source]['nom_structure']}2"])

In [7]:
id_struct.progress_apply(lambda row: get_structure(row,source,cached_data,sources[source]['nom_structure'],sources[source]['ville'],sources[source]['pays'],sources[source]['code_projet'],False), axis=1) 
write_cache(cached_data,f"./DATA/{source}/caches/cached_{source}_data.pkl")

100%|██████████| 965/965 [00:03<00:00, 279.42it/s]


In [8]:
len(cached_data)

970

In [9]:
id_struct['id_structure_matcher']=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: cached_data[x])

C:\Users\haallat\AppData\Local\Temp\ipykernel_12088\3780844401.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_struct['id_structure_matcher']=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: cached_data[x])


In [10]:
id_struct

,Code convention,code convention homogénéisé,Acronyme,Nom Responsable Scientifique,Prénom Responsable Scientifique,Organisme,CP,Ville,Pays,Organisme2,id_structure_matcher
0,2006/1/2,ANSES-06-EST-002,PAISIM,Bard,Denis,LERES (Laboratoire d’Étude et de Recherche en ...,35403,Rennes,France,lereboratoireurechercheenvironnementante,None
1,2006/1/3,ANSES-06-EST-003,NaN,Droz,Olivier,IST Institut universitaire romand de Santé au ...,NaN,Lausanne,Suisse,itintitutuniveritaireromandanteautravail,None
2,2006/1/6,ANSES-06-EST-006,NaN,Frimat,Paul,Faculté de Médecine - Université Lille 2 - Lab...,59037,Lille,France,facultemecineuniveritelle2boratoiremecineantea...,[199814194S]
3,2006/1/7,ANSES-06-EST-007,NaN,Sauvain,Jean-Jacques,Groupe Particule et santé - Pôle Santé des Tra...,NaN,Lausanne,Suisse,groupeiculeantepoleantravailleur,None
4,2006/1/8,ANSES-06-EST-008,CATHY,Guénel,Pascal,U754 - Epidémiologie environnementale des cancers,94807,Villejuif,France,u754epimiologieenvironnementalecancer,[200616425G]
...,...,...,...,...,...,...,...,...,...,...,...
1727,EST/2017/2 RF/15,ANSES-17-RF-15,Coriolis,Conil,Emmanuelle,Agence nationale des fréquences,94704,Maisons-Alfort,France,agencenationalefrequence,None
1729,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Ghaleh-Marzban,Bijan,Plateforme des ressources biologiques,94000,Créteil,France,plateformereourcebiologique,None
1731,2020/01/245,ANSES-20-EST-245,PESTINTER,Bec,Alexandre,63178,Aubière,France,NaN,63178,None
1732,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Montagne,Olivier,Centre d'investigation Clinique 006 Créteil,94000,Créteil,France,centrenvigationclinique006creil,None


In [11]:
id_struct=id_struct.reset_index()
del id_struct['index']
id_struct.to_excel(f"./DATA/{source}/df_partenaires.xlsx")
id_struct.to_json(f"./DATA/{source}/df_partenaires.json")

In [12]:
id_struct=pd.read_json(f"./DATA/{source}/df_partenaires.json")
id_struct=id_struct[[sources[source]['nom_structure'],'id_structure_matcher']]
id_struct[f"{sources[source]['nom_structure']}2"]=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))

df_partenaires[f"{sources[source]['nom_structure']}2"]=df_partenaires.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
df_partenaires_struct=pd.merge(df_partenaires,id_struct[[f"{sources[source]['nom_structure']}2",'id_structure_matcher']], on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_struct

,Code convention,code convention homogénéisé,Acronyme,Nom Responsable Scientifique,Prénom Responsable Scientifique,Organisme,CP,Ville,Pays,Organisme2,id_structure_matcher
0,2006/1/2,ANSES-06-EST-002,PAISIM,Bard,Denis,LERES (Laboratoire d’Étude et de Recherche en ...,35403,Rennes,France,lereboratoireurechercheenvironnementante,None
1,2006/1/3,ANSES-06-EST-003,NaN,Droz,Olivier,IST Institut universitaire romand de Santé au ...,NaN,Lausanne,Suisse,itintitutuniveritaireromandanteautravail,None
2,2006/1/6,ANSES-06-EST-006,NaN,Frimat,Paul,Faculté de Médecine - Université Lille 2 - Lab...,59037,Lille,France,facultemecineuniveritelle2boratoiremecineantea...,[199814194S]
3,2006/1/7,ANSES-06-EST-007,NaN,Sauvain,Jean-Jacques,Groupe Particule et santé - Pôle Santé des Tra...,NaN,Lausanne,Suisse,groupeiculeantepoleantravailleur,None
4,2006/1/8,ANSES-06-EST-008,CATHY,Guénel,Pascal,U754 - Epidémiologie environnementale des cancers,94807,Villejuif,France,u754epimiologieenvironnementalecancer,[200616425G]
...,...,...,...,...,...,...,...,...,...,...,...
1732,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Montagne,Olivier,Centre d'investigation Clinique 006 Créteil,94000,Créteil,France,centrenvigationclinique006creil,None
1733,2013/01/216,ANSES-13-EST-216,PATER,Malherbe,Laure,Ineris,60550,Verneuil-en-Halatte,France,ineri,None
1734,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Paradis,Valérie,NaN,92110,Clichy,France,nan,None
1735,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Boczkowski,Jorge,Université Paris 12 - Faculté de Médecine,94000,Créteil,France,univeritepari12facultemecine,None


In [13]:
#compléter les données avec scanR
url_scanr='https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/scanR/projects.json'
requete_scanR = requests.get(url_scanr)
page_scanR= requete_scanR.json()
df_scanR=pd.DataFrame(page_scanR)
scanR=df_scanR.explode('participants').loc[:,['id','participants']]
scanR=scanR.rename(columns={'id':'id_anr'})
scanR['index']=[x for x in range(len(scanR))]
scanR=scanR.set_index('index')
scanR['id_structure_scanr']=scanR['participants'].apply(lambda x: x.get(str('structure')) if isinstance(x, dict) else None )
scanR['nom_struct']=scanR['participants'].apply(lambda x: get_scanR_structure(x))
del scanR['participants']
scanR_nettoye=scanR.drop_duplicates(subset='nom_struct')
scanR_nettoye[f"{sources[source]['nom_structure']}2"]=scanR_nettoye.loc[:,'nom_struct'].apply(lambda x: replace_all(str(x).lower()))
scanR_nettoye=scanR_nettoye[['id_structure_scanr',f"{sources[source]['nom_structure']}2"]]
scanR_nettoye=scanR_nettoye.drop_duplicates(subset=f"{sources[source]['nom_structure']}2")

C:\Users\haallat\AppData\Local\Temp\ipykernel_12088\450009379.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scanR_nettoye[f"{sources[source]['nom_structure']}2"]=scanR_nettoye.loc[:,'nom_struct'].apply(lambda x: replace_all(str(x).lower()))


In [14]:
df_partenaires_struct=pd.merge(df_partenaires_struct,scanR_nettoye, on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_struct

,Code convention,code convention homogénéisé,Acronyme,Nom Responsable Scientifique,Prénom Responsable Scientifique,Organisme,CP,Ville,Pays,Organisme2,id_structure_matcher,id_structure_scanr
0,2006/1/2,ANSES-06-EST-002,PAISIM,Bard,Denis,LERES (Laboratoire d’Étude et de Recherche en ...,35403,Rennes,France,lereboratoireurechercheenvironnementante,None,NaN
1,2006/1/3,ANSES-06-EST-003,NaN,Droz,Olivier,IST Institut universitaire romand de Santé au ...,NaN,Lausanne,Suisse,itintitutuniveritaireromandanteautravail,None,NaN
2,2006/1/6,ANSES-06-EST-006,NaN,Frimat,Paul,Faculté de Médecine - Université Lille 2 - Lab...,59037,Lille,France,facultemecineuniveritelle2boratoiremecineantea...,[199814194S],NaN
3,2006/1/7,ANSES-06-EST-007,NaN,Sauvain,Jean-Jacques,Groupe Particule et santé - Pôle Santé des Tra...,NaN,Lausanne,Suisse,groupeiculeantepoleantravailleur,None,NaN
4,2006/1/8,ANSES-06-EST-008,CATHY,Guénel,Pascal,U754 - Epidémiologie environnementale des cancers,94807,Villejuif,France,u754epimiologieenvironnementalecancer,[200616425G],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1732,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Montagne,Olivier,Centre d'investigation Clinique 006 Créteil,94000,Créteil,France,centrenvigationclinique006creil,None,NaN
1733,2013/01/216,ANSES-13-EST-216,PATER,Malherbe,Laure,Ineris,60550,Verneuil-en-Halatte,France,ineri,None,NaN
1734,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Paradis,Valérie,NaN,92110,Clichy,France,nan,NaN,NaN
1735,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Boczkowski,Jorge,Université Paris 12 - Faculté de Médecine,94000,Créteil,France,univeritepari12facultemecine,None,NaN


In [15]:
#######fichier avec les identifiants structures rettrouvés à la main par Emmanuel ==> 'code'
scanr_structures=pd.read_excel('scanr_partenaires_non_identifies.xlsx')
scanr_structures[f"{sources[source]['nom_structure']}2"]=scanr_structures.loc[:,'Nom'].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
scanr_structures=scanr_structures[[f"{sources[source]['nom_structure']}2",'code']]
scanr_structures=scanr_structures.dropna().drop_duplicates(subset=f"{sources[source]['nom_structure']}2")
df_partenaires_complet=pd.merge(df_partenaires_struct,scanr_structures, on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_complet

c:\Users\haallat\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Code convention,code convention homogénéisé,Acronyme,Nom Responsable Scientifique,Prénom Responsable Scientifique,Organisme,CP,Ville,Pays,Organisme2,id_structure_matcher,id_structure_scanr,code
0,2006/1/2,ANSES-06-EST-002,PAISIM,Bard,Denis,LERES (Laboratoire d’Étude et de Recherche en ...,35403,Rennes,France,lereboratoireurechercheenvironnementante,None,NaN,201222588E
1,2006/1/3,ANSES-06-EST-003,NaN,Droz,Olivier,IST Institut universitaire romand de Santé au ...,NaN,Lausanne,Suisse,itintitutuniveritaireromandanteautravail,None,NaN,04mcdza51
2,2006/1/6,ANSES-06-EST-006,NaN,Frimat,Paul,Faculté de Médecine - Université Lille 2 - Lab...,59037,Lille,France,facultemecineuniveritelle2boratoiremecineantea...,[199814194S],NaN,NaN
3,2006/1/7,ANSES-06-EST-007,NaN,Sauvain,Jean-Jacques,Groupe Particule et santé - Pôle Santé des Tra...,NaN,Lausanne,Suisse,groupeiculeantepoleantravailleur,None,NaN,04mcdza51
4,2006/1/8,ANSES-06-EST-008,CATHY,Guénel,Pascal,U754 - Epidémiologie environnementale des cancers,94807,Villejuif,France,u754epimiologieenvironnementalecancer,[200616425G],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Montagne,Olivier,Centre d'investigation Clinique 006 Créteil,94000,Créteil,France,centrenvigationclinique006creil,None,NaN,200120111K
1733,2013/01/216,ANSES-13-EST-216,PATER,Malherbe,Laure,Ineris,60550,Verneuil-en-Halatte,France,ineri,None,None,0602098E
1734,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Paradis,Valérie,NaN,92110,Clichy,France,nan,NaN,NaN,x
1735,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Boczkowski,Jorge,Université Paris 12 - Faculté de Médecine,94000,Créteil,France,univeritepari12facultemecine,None,NaN,vb71K


In [16]:
if 'finess' in list(df_partenaires.columns):
    finess_siret=pd.read_json("finess_siret-siege.json")
    df_partenaires_complet=pd.merge(df_partenaires_complet,finess_siret,how='left', on='finess')

In [17]:
df_partenaires_complet[sources[source]['identifiants_preferes_structure']]

,id_structure_matcher,id_structure_scanr,code
0,None,NaN,201222588E
1,None,NaN,04mcdza51
2,[199814194S],NaN,NaN
3,None,NaN,04mcdza51
4,[200616425G],NaN,NaN
...,...,...,...
1732,None,None,200120111K
1733,None,None,0602098E
1734,None,None,x
1735,None,None,vb71K


In [18]:
df_partenaires_complet.columns

Index(['Code convention', 'code convention homogénéisé', 'Acronyme',
       'Nom Responsable Scientifique', 'Prénom Responsable Scientifique',
       'Organisme', 'CP ', 'Ville', 'Pays', 'Organisme2',
       'id_structure_matcher', 'id_structure_scanr', 'code'],
      dtype='object')

In [19]:
df_partenaires_complet['id_structure']=df_partenaires_complet.apply(lambda row: get_id(row,sources[source]['identifiants_preferes_structure']), axis=1)
df_partenaires_complet

,Code convention,code convention homogénéisé,Acronyme,Nom Responsable Scientifique,Prénom Responsable Scientifique,Organisme,CP,Ville,Pays,Organisme2,id_structure_matcher,id_structure_scanr,code,id_structure
0,2006/1/2,ANSES-06-EST-002,PAISIM,Bard,Denis,LERES (Laboratoire d’Étude et de Recherche en ...,35403,Rennes,France,lereboratoireurechercheenvironnementante,None,NaN,201222588E,201222588E
1,2006/1/3,ANSES-06-EST-003,NaN,Droz,Olivier,IST Institut universitaire romand de Santé au ...,NaN,Lausanne,Suisse,itintitutuniveritaireromandanteautravail,None,NaN,04mcdza51,04mcdza51
2,2006/1/6,ANSES-06-EST-006,NaN,Frimat,Paul,Faculté de Médecine - Université Lille 2 - Lab...,59037,Lille,France,facultemecineuniveritelle2boratoiremecineantea...,[199814194S],NaN,NaN,[199814194S]
3,2006/1/7,ANSES-06-EST-007,NaN,Sauvain,Jean-Jacques,Groupe Particule et santé - Pôle Santé des Tra...,NaN,Lausanne,Suisse,groupeiculeantepoleantravailleur,None,NaN,04mcdza51,04mcdza51
4,2006/1/8,ANSES-06-EST-008,CATHY,Guénel,Pascal,U754 - Epidémiologie environnementale des cancers,94807,Villejuif,France,u754epimiologieenvironnementalecancer,[200616425G],NaN,NaN,[200616425G]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Montagne,Olivier,Centre d'investigation Clinique 006 Créteil,94000,Créteil,France,centrenvigationclinique006creil,None,NaN,200120111K,200120111K
1733,2013/01/216,ANSES-13-EST-216,PATER,Malherbe,Laure,Ineris,60550,Verneuil-en-Halatte,France,ineri,None,None,0602098E,0602098E
1734,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Paradis,Valérie,NaN,92110,Clichy,France,nan,NaN,NaN,x,x
1735,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Boczkowski,Jorge,Université Paris 12 - Faculté de Médecine,94000,Créteil,France,univeritepari12facultemecine,None,NaN,vb71K,vb71K


In [20]:
#df_partenaires_complet[pd.isna(df_partenaires_complet.id_structure)]
df_partenaires_complet.loc[(pd.isna(df_partenaires_complet['id_structure']))|(str(df_partenaires_complet['id_structure'])=='None')|(str(df_partenaires_complet['id_structure'])=='nan')]

,Code convention,code convention homogénéisé,Acronyme,Nom Responsable Scientifique,Prénom Responsable Scientifique,Organisme,CP,Ville,Pays,Organisme2,id_structure_matcher,id_structure_scanr,code,id_structure
1504,EST-2012/2/044 (RF),ANSES-12-EST-044,ACTe,Peyman,Azadeh,Health Protection Agency\nCentre for Radiation...,NaN,NaN,Grande Bretagne,healthprotectionagency\ncentreradiationchemica...,None,NaN,NaN,None


In [21]:
#df_partenaires_complet.to_excel(f"./DATA/{source}/df_partenaires_id_structures.xlsx")
df_partenaires_complet.to_json(f"./DATA/{source}/df_partners_id_structures.json")

In [23]:
df_partenaires_complet=pd.read_json(f"./DATA/{source}/df_partners_id_structures.json")

In [24]:
########récupération des structures sans identifiants pour les donner à Emmanuel
identifiants_a_remplir=df_partenaires_complet.loc[(pd.isna(df_partenaires_complet['id_structure']))|(str(df_partenaires_complet['id_structure'])=='None')|(str(df_partenaires_complet['id_structure'])=='nan')]
identifiants_a_remplir
identifiants_a_remplir=identifiants_a_remplir.drop_duplicates(subset=f"{sources[source]['nom_structure']}2")
identifiants_a_remplir=identifiants_a_remplir.reset_index()
del identifiants_a_remplir['index']

In [25]:
if sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] in list(identifiants_a_remplir.columns) and sources[source]['adresse'] not in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['ville'],sources[source]['pays']]]
elif sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] in list(identifiants_a_remplir.columns) and sources[source]['adresse'] in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['adresse'],sources[source]['ville'],sources[source]['pays']]]
elif sources[source]['region'] in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['region']]]
elif sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] not in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['ville']]]
identifiants_a_remplir

,Organisme,Ville,Pays
0,Health Protection Agency\nCentre for Radiation...,None,Grande Bretagne


In [26]:
identifiants_a_remplir.to_excel(f"./structures_manquantes/partenaires_non_identifies_{source}.xlsx", index=False)

# Matcher des chercheurs

In [27]:
df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_structures.json")

In [ ]:
if len([x for x in ['nom', 'prenom'] if x in list(sources[source].keys())])==2:
    df_partenaires['id_personne']=df_partenaires.progress_apply(lambda row: get_person(row, cached_data_persons,sources[source]['nom'],sources[source]['prenom']), axis=1)
    #df_partenaires.to_excel(f"./DATA/{source}/df_partners_id_personne.xlsx")
    df_partenaires.to_json(f"./DATA/{source}/df_partners_id_person.json")

  0%|          | 6/1737 [00:00<02:28, 11.67it/s]c:\Users\haallat\Anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
  1%|          | 20/1737 [00:03<12:59,  2.20it/s]

no exact fullname match for Morin Jean-Paul vs ['trevanian', 'trevanian']


  1%|▏         | 25/1737 [00:05<10:02,  2.84it/s]

no exact fullname match for Clavel Jacqueline vs ['jacqueline clavel marinacce', 'clavel marinacce jacqueline']


  2%|▏         | 27/1737 [00:05<07:20,  3.88it/s]

...

 99%|█████████▉| 1719/1737 [06:00<00:06,  2.87it/s]

no exact fullname match for Petit Fabienne vs ['fabienne arias petit', 'arias petit fabienne']
no exact fullname match for Petit Fabienne vs ['fabienne petit jean', 'petit jean fabienne']


100%|█████████▉| 1731/1737 [06:02<00:01,  5.20it/s]

no exact fullname match for Ghaleh-Marzban Bijan vs ['bijan ghaleh', 'ghaleh bijan']


100%|██████████| 1737/1737 [06:03<00:00,  4.77it/s]


In [29]:
len(cached_data_persons)

800

In [30]:
write_cache(cached_data_persons,f"./DATA/{source}/caches/cached_{source.lower()}_data_persons.pkl")

In [31]:
df_partenaires['id_personne']=df_partenaires.progress_apply(lambda row: get_person(row, cached_data_persons,sources[source]['nom'],sources[source]['prenom']), axis=1)

  1%|          | 14/1737 [00:00<01:40, 17.21it/s]c:\Users\haallat\Anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
  1%|          | 20/1737 [00:02<03:33,  8.03it/s]

no exact fullname match for Morin Jean-Paul vs ['trevanian', 'trevanian']


  2%|▏         | 27/1737 [00:03<04:15,  6.71it/s]

no exact fullname match for Gilot-Fromont Emmanuelle vs ['emmanuelle fromont', 'fromont emmanuelle']


  2%|▏         | 29/1737 [00:03<04:25,  6.42it/s]

...

 99%|█████████▉| 1718/1737 [03:27<00:04,  4.75it/s]

no exact fullname match for Gascoin Géraldine vs ['geraldine gascoin lachambre', 'gascoin lachambre geraldine']
no exact fullname match for Petit Fabienne vs ['fabienne arias petit', 'arias petit fabienne']
no exact fullname match for Petit Fabienne vs ['fabienne petit jean', 'petit jean fabienne']


100%|█████████▉| 1731/1737 [03:28<00:00,  7.87it/s]

no exact fullname match for Ghaleh-Marzban Bijan vs ['bijan ghaleh', 'ghaleh bijan']


100%|██████████| 1737/1737 [03:29<00:00,  8.30it/s]


In [32]:
df_partenaires.columns

Index(['Code convention', 'code convention homogénéisé', 'Acronyme',
       'Nom Responsable Scientifique', 'Prénom Responsable Scientifique',
       'Organisme', 'CP ', 'Ville', 'Pays', 'Organisme2',
       'id_structure_matcher', 'id_structure_scanr', 'code', 'id_structure',
       'id_personne'],
      dtype='object')

In [ ]:
if sources[source]['id_ORCID'] in list(df_partenaires.columns):
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_person.json")
    df_partenaires['idref_ORCID']=df_partenaires.progress_apply(lambda row: orcid_to_idref(row,cached_data_orcid,sources[source]['id_ORCID'],Authorization_ORCID), axis=1)
    #write_cache(cached_data_orcid,f"./DATA/{source}/caches/cached_{source.lower()}_data_orcid.pkl")
    #df_partenaires.to_excel(f"./DATA/{source}/df_partners_id_person_ORCID.xlsx")
    df_partenaires.to_json(f"./DATA/{source}/df_partners_id_person_ORCID.json")

In [35]:
len(cached_data_orcid)

0

# ENVOI DES PROJETS SUR SCANR

In [39]:
if len(sources[source]['identifiants_preferes_personne'])==2:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_person_ORCID.json")
elif len(sources[source]['identifiants_preferes_personne'])==1:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_person.json")
else:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_structures.json")

In [40]:
df_partenaires.loc[df_partenaires.id_structure.apply(lambda x :isinstance(x,list)),'id_structure']=df_partenaires.loc[df_partenaires.id_structure.apply(lambda x :isinstance(x,list)),'id_structure'].apply(lambda y: y[0])

In [41]:
if len([x for x in ['nom', 'prenom'] if x in list(sources[source].keys())])==2:
    df_partenaires['id_person']=df_partenaires.apply(lambda row: get_id(row,sources[source]['identifiants_preferes_personne']), axis=1)
    df_partenaires['persons']=df_partenaires.progress_apply(lambda row: persons(row,sources[source]['prenom'],sources[source]['nom']) ,axis=1)
else:
    df_partenaires['persons']=np.nan

100%|██████████| 1737/1737 [00:00<00:00, 11757.80it/s]


In [42]:
if source != 'SIRANO':
    df_partenaires=df_partenaires.groupby([sources[source]['code_projet']]).agg({'persons': lambda x: [ y for y in x.tolist() if pd.isna(y)==False]}, dropna=False).reset_index()
else:
    df_projets=df_partenaires.groupby([sources[source]['code_projet'], sources[source]['annee'], sources[source]['acronyme'],sources[source]['titre'],sources[source]['budget']], dropna=False).agg({'persons': lambda x: [ y for y in x.tolist() if pd.isna(y)==False]}, dropna=False)

In [43]:
# amener les projets depuis le site 
if source=='ANR':
    page_projets_10 = requests.get(sources[source]['url_projects']).json()
    colonnes_projets_10 = page_projets_10['columns']
    donnees_projets_10 = page_projets_10['data']
    df_projets=pd.DataFrame(data=donnees_projets_10,columns=colonnes_projets_10)
elif source=='IRESP':
    df_projets1=pd.read_csv(sources[source]['url_projects1'] ,sep=";", encoding='UTF-8')
    df_projets2=pd.read_csv(sources[source]['url_projects2'] ,sep=";", encoding='UTF-8')
    df_projets=pd.concat([df_projets1,df_projets2])
    df_projets.loc[pd.isna(df_projets['Titre_du_projet_FR']),'Titre_du_projet_FR']=df_projets.loc[pd.isna(df_projets['Titre_du_projet_FR']),'Titre_du_projet']
elif source!='SIRANO':
    df_projets=pd.read_csv(sources[source]['url_projects'] ,sep=";", encoding='ISO-8859-1')

df_projets=df_projets.reset_index()
del df_projets['index']

In [44]:
if source!='SIRANO':
    df_projets=pd.merge(df_projets,df_partenaires,on=sources[source]['code_projet'], how='left')
else :
    df_projets['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[sources[source]['annee']]}-{row[sources[source]['acronyme']]}" , axis=1)
    del df_projets['code_projet']
    sources[source]['code_projet']='id'

df_projets['type']=source
df_projets['name']=df_projets.progress_apply(lambda row: projects(row,sources[source]['titre_fr'],sources[source]['titre_en']) ,axis=1)
df_projets['description']=df_projets.progress_apply(lambda row: projects(row,sources[source]['resume_fr'],sources[source]['resume_en']) ,axis=1)
df_projets.loc[:,sources[source]['budget']]=df_projets.loc[:,sources[source]['budget']].apply(lambda x : float(str(x).replace('.0','').replace('.00','').replace(' ','').replace(',','.').replace('€','').replace('\x80','')))
df_projets=df_projets.rename(columns={sources[source]['annee']:'year',sources[source]['acronyme']:'acronym',
                                      sources[source]['budget']:'budget_financed',sources[source]['code_projet']:'id'})
df_projets=df_projets[['id','type','name','description','acronym','year','budget_financed','persons']]

100%|██████████| 592/592 [00:00<00:00, 16489.10it/s]


In [45]:
dict_row=df_projets.iloc[0,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
dict_row2

{'id': 'ANSES-06-EST-002',
 'type': 'ANSES',
 'name': {'fr': 'Projet PAISIM : Pollution Atmosphérique, Inégalités Sociales et Infarctus du Myocarde',
  'en': 'The PAISIM project: Air Pollution, Social Inequalities & Myocardial Infarction'},
 'description': {'fr': 'Depuis une quinzaine d\x92années, plusieurs équipes se sont intéressées aux liens existant entre le polymorphisme de l\x92ALAD G177C (acide delta-aminolévulinique déshydratase) (rs1800435) et l\x92intoxication par le plomb. Les études publiées ne sont pas concordantes quant aux liens existants entre ce polymorphisme et les effets du plomb sur la fonction rénale. L\x92opportunité nous est donnée d\x92analyser ces liens sur une cohorte d\x92ex-salariés d\x92une fonderie de plomb et de zinc de la Région Nord \x96 Pas de Calais, qui bénéficient depuis 2003 d\x92un suivi clinique et biologique post-expositionnel. Ce suivi va être prolongé jusqu\x92en 2009, dans le cadre d\x92une étude scientifique visant à étudier les facteurs de 

In [ ]:
#envoi
err=[]
for i,row in df_projets.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
    try:
       r=requests.post('http://185.161.45.213/projects/projects', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

In [ ]:
pd.Series([x.get('issues').get('id') for x in err]).drop_duplicates().tolist() 

pour mettre à jour

In [46]:
###AAATTTENTION CHANGER A CHAQUE FOIS LE TYPE
nbr_page=int(requests.get('http://185.161.45.213/projects/projects?where={"type":"ANSES"}&projection={"id":1}&max_results=500&page=1', headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])

list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/projects?where={"type":"ANSES"}&projection={"id":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        print("k",k)
        list_ids.append(page['data'][k]['id'])
    
projets_a_ajouter=[x for x in list(df_projets['id']) if x not in list_ids]

projets_a_retirer=[x for x in list_ids if x not in list(df_projets['id'])]

df_projets = df_projets[df_projets['id'].apply(lambda x: x in projets_a_ajouter)]

page 1
k 0
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
k 26
k 27
k 28
k 29
k 30
k 31
k 32
k 33
k 34
k 35
k 36
k 37
k 38
k 39
k 40
k 41
k 42
k 43
k 44
k 45
k 46
k 47
k 48
k 49
k 50
k 51
k 52
k 53
k 54
k 55
k 56
k 57
k 58
k 59
k 60
k 61
k 62
k 63
k 64
k 65
k 66
k 67
k 68
k 69
k 70
k 71
k 72
k 73
k 74
k 75
k 76
k 77
k 78
k 79
k 80
k 81
k 82
k 83
k 84
k 85
k 86
k 87
k 88
k 89
k 90
k 91
k 92
k 93
k 94
k 95
k 96
k 97
k 98
k 99
k 100
k 101
k 102
k 103
k 104
k 105
k 106
k 107
k 108
k 109
k 110
k 111
k 112
k 113
k 114
k 115
k 116
k 117
k 118
k 119
k 120
k 121
k 122
k 123
k 124
k 125
k 126
k 127
k 128
k 129
k 130
k 131
k 132
k 133
k 134
k 135
k 136
k 137
k 138
k 139
k 140
k 141
k 142
k 143
k 144
k 145
k 146
k 147
k 148
k 149
k 150
k 151
k 152
k 153
k 154
k 155
k 156
k 157
k 158
k 159
k 160
k 161
k 162
k 163
k 164
k 165
k 166
k 167
k 168
k 169
k 170
k 171
k 172
k 173
k 174
k 175
k 176
k 177
k 178
k 179
k 180
k 181
k 182
k 183

In [47]:
len(projets_a_ajouter)

0

In [ ]:
len(df_projets[df_projets['id'].apply(lambda x: x in projets_a_ajouter)])

In [ ]:
len(df_projets[df_projets['id'].apply(lambda x: x in projets_a_retirer)])

In [ ]:
dict_row=df_projets.iloc[0,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
dict_row2

In [ ]:
err=[]
for i,row in df_projets.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
    try:
       r=requests.post('http://185.161.45.213/projects/projects', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

In [ ]:
len(projets_a_ajouter)

Modifications

In [ ]:
nbr_page=int(requests.get('http://185.161.45.213/projects/projects?where={"type":"ANR"}&projection={"id":1,"year":1}&max_results=500'+f"&page={1}", headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])
nbr_page

In [ ]:
list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/projects?where={%22type%22:%22ANR%22}&projection={%22id%22:1,%22year%22:1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        list_ids.append(page['data'][k]['year'])

In [ ]:
max(list_ids)

In [ ]:
df_projets = df_projets[df_projets['id'].apply(lambda x: x in list_ids)]

In [ ]:
"""
en principe, il faut que je mette à jour tous les projets 
qui ne sont pas clos avec les titres résumés et persons, mais 
peut etre qu'on peut mettre à jour automatiquement les projets
des 3 dernières années ? 

"""

err=[]
for id in df_projets.iterrows() :
    url = f"http://185.161.45.213/projects/projects/{id}"
    project=requests.get(url, headers={'Authorization': Authorization}).json()
    head = {"Authorization": Authorization, "If-Match": project['etag'], "Content-Type": "application/json"}

    r = requests.patch(url, json = {"year": 2023}, headers=head)
    res= r.json()
    if res.get('status')=='ERR':
        err.append(res)
        if res.get('error').get('code')!=422:
            print(err)
            pp.pprint(err)


# ENVOI DES PARTENAIRES SUR SCANR

In [48]:
df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_structures.json")

In [49]:
df_partenaires.columns

Index(['Code convention', 'code convention homogénéisé', 'Acronyme',
       'Nom Responsable Scientifique', 'Prénom Responsable Scientifique',
       'Organisme', 'CP ', 'Ville', 'Pays', 'Organisme2',
       'id_structure_matcher', 'id_structure_scanr', 'code', 'id_structure'],
      dtype='object')

In [50]:
### ATTENTION, vérifier que les projets sirano sont dans des structures françaises
if source=='IRESP':
    df_partenaires[sources[source]['pays']]=df_partenaires.loc[:,sources[source]['ville']].apply(lambda x: x.split('(')[1].replace(')','') if x.find('(')>=0 else 'France')
    df_partenaires.loc[:,sources[source]['ville']]=df_partenaires.loc[:,sources[source]['ville']].apply(lambda x: x.split('(')[0] if x.find('(')>=0 else x)
  
df_partenaires['address']=df_partenaires.apply(lambda row: address(row,sources[source]['pays'],sources[source]['ville'],source), axis=1)
df_partenaires.loc[:,'id_structure']=df_partenaires.loc[:,'id_structure'].apply(lambda x: x[0] if isinstance(x,list) else x )

In [51]:
df_partenaires

,Code convention,code convention homogénéisé,Acronyme,Nom Responsable Scientifique,Prénom Responsable Scientifique,Organisme,CP,Ville,Pays,Organisme2,id_structure_matcher,id_structure_scanr,code,id_structure,address
0,2006/1/2,ANSES-06-EST-002,PAISIM,Bard,Denis,LERES (Laboratoire d’Étude et de Recherche en ...,35403,Rennes,France,lereboratoireurechercheenvironnementante,None,None,201222588E,201222588E,"{'city': 'Rennes', 'country': 'France'}"
1,2006/1/3,ANSES-06-EST-003,None,Droz,Olivier,IST Institut universitaire romand de Santé au ...,None,Lausanne,Suisse,itintitutuniveritaireromandanteautravail,None,None,04mcdza51,04mcdza51,"{'city': 'Lausanne', 'country': 'Suisse'}"
2,2006/1/6,ANSES-06-EST-006,None,Frimat,Paul,Faculté de Médecine - Université Lille 2 - Lab...,59037,Lille,France,facultemecineuniveritelle2boratoiremecineantea...,[199814194S],None,None,199814194S,"{'city': 'Lille', 'country': 'France'}"
3,2006/1/7,ANSES-06-EST-007,None,Sauvain,Jean-Jacques,Groupe Particule et santé - Pôle Santé des Tra...,None,Lausanne,Suisse,groupeiculeantepoleantravailleur,None,None,04mcdza51,04mcdza51,"{'city': 'Lausanne', 'country': 'Suisse'}"
4,2006/1/8,ANSES-06-EST-008,CATHY,Guénel,Pascal,U754 - Epidémiologie environnementale des cancers,94807,Villejuif,France,u754epimiologieenvironnementalecancer,[200616425G],None,None,200616425G,"{'city': 'Villejuif', 'country': 'France'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Montagne,Olivier,Centre d'investigation Clinique 006 Créteil,94000,Créteil,France,centrenvigationclinique006creil,None,None,200120111K,200120111K,"{'city': 'Créteil', 'country': 'France'}"
1733,2013/01/216,ANSES-13-EST-216,PATER,Malherbe,Laure,Ineris,60550,Verneuil-en-Halatte,France,ineri,None,None,0602098E,0602098E,"{'city': 'Verneuil-en-Halatte', 'country': 'Fr..."
1734,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Paradis,Valérie,None,92110,Clichy,France,nan,None,None,x,x,"{'city': 'Clichy', 'country': 'France'}"
1735,EST-2009/1/101,ANSES-09-EST-101,MB2PROF,Boczkowski,Jorge,Université Paris 12 - Faculté de Médecine,94000,Créteil,France,univeritepari12facultemecine,None,None,vb71K,vb71K,"{'city': 'Créteil', 'country': 'France'}"


In [52]:
if source in ['ANSES','SIRANO']:
    df_partenaires['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[str(sources[source]['nom_structure'])+'2']}-{row[sources[source]['nom']]}-{row[sources[source]['prenom']]}" , axis=1)
if source =='REG_IDF':
    df_partenaires['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[str(sources[source]['nom_structure'])+'2']}-{row['entite_role']}" , axis=1)
df_partenaires['address']=df_partenaires.apply(lambda row: address(row,sources[source]['pays'],sources[source]['ville'],source), axis=1)
df_partenaires=df_partenaires.rename(columns={sources[source]['nom_structure']: 'name', sources[source]['code_projet']: 'project_id', 'id_structure':'participant_id','Projet.Partenaire.Code_Decision_ANR':'id'})
df_partenaires=df_partenaires[['name','id','project_id','participant_id','address']]
df_partenaires['project_type']=source
df_partenaires['participant_id']=df_partenaires.loc[:,'participant_id'].apply(lambda x: str(x[0]).replace('.0','') if isinstance(x,list) else str(x).split(';')[0].replace('.0',''))
df_partenaires=df_partenaires[['id','project_id', 'project_type', 'participant_id', 'name','address']]
df_partenaires['name'] = df_partenaires['name'].astype(str)
df_partenaires

,id,project_id,project_type,participant_id,name,address
0,ANSES-06-EST-002-lereboratoireurechercheenviro...,ANSES-06-EST-002,ANSES,201222588E,LERES (Laboratoire d’Étude et de Recherche en ...,"{'city': 'Rennes', 'country': 'France'}"
1,ANSES-06-EST-003-itintitutuniveritaireromandan...,ANSES-06-EST-003,ANSES,04mcdza51,IST Institut universitaire romand de Santé au ...,"{'city': 'Lausanne', 'country': 'Suisse'}"
2,ANSES-06-EST-006-facultemecineuniveritelle2bor...,ANSES-06-EST-006,ANSES,199814194S,Faculté de Médecine - Université Lille 2 - Lab...,"{'city': 'Lille', 'country': 'France'}"
3,ANSES-06-EST-007-groupeiculeantepoleantravaill...,ANSES-06-EST-007,ANSES,04mcdza51,Groupe Particule et santé - Pôle Santé des Tra...,"{'city': 'Lausanne', 'country': 'Suisse'}"
4,ANSES-06-EST-008-u754epimiologieenvironnementa...,ANSES-06-EST-008,ANSES,200616425G,U754 - Epidémiologie environnementale des cancers,"{'city': 'Villejuif', 'country': 'France'}"
...,...,...,...,...,...,...
1732,ANSES-09-EST-101-centrenvigationclinique006cre...,ANSES-09-EST-101,ANSES,200120111K,Centre d'investigation Clinique 006 Créteil,"{'city': 'Créteil', 'country': 'France'}"
1733,ANSES-13-EST-216-ineri-Malherbe-Laure,ANSES-13-EST-216,ANSES,0602098E,Ineris,"{'city': 'Verneuil-en-Halatte', 'country': 'Fr..."
1734,ANSES-09-EST-101-nan-Paradis-Valérie,ANSES-09-EST-101,ANSES,x,None,"{'city': 'Clichy', 'country': 'France'}"
1735,ANSES-09-EST-101-univeritepari12facultemecine-...,ANSES-09-EST-101,ANSES,vb71K,Université Paris 12 - Faculté de Médecine,"{'city': 'Créteil', 'country': 'France'}"


In [53]:
df_partenaires[df_partenaires.duplicated(subset=['id'])]

,id,project_id,project_type,participant_id,name,address


In [ ]:
dict_row=df_partenaires.iloc[1,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
dict_row2

In [ ]:
len(df_partenaires)

In [ ]:
err=[]
for i,row in df_partenaires.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    try:
       r=requests.post('http://185.161.45.213/projects/participations', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           print(i)
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

Mise à jour

In [54]:
nbr_page=int(requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANSES"}&projection={"id":1}&max_results=500&page=1', headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])

list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANSES"}&projection={"id":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        print("k",k)
        list_ids.append(page['data'][k]['id'])
    
projets_a_ajouter=[x for x in list(df_partenaires['id'].drop_duplicates()) if x not in list(pd.Series(list_ids).drop_duplicates())]

projets_a_retirer=[x for x in list_ids if x not in list(df_partenaires['id'])]

df_partenaires = df_partenaires[df_partenaires['id'].apply(lambda x: x in projets_a_ajouter)]


page 1
k 0
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
k 26
k 27
k 28
k 29
k 30
k 31
k 32
k 33
k 34
k 35
k 36
k 37
k 38
k 39
k 40
k 41
k 42
k 43
k 44
k 45
k 46
k 47
k 48
k 49
k 50
k 51
k 52
k 53
k 54
k 55
k 56
k 57
k 58
k 59
k 60
k 61
k 62
k 63
k 64
k 65
k 66
k 67
k 68
k 69
k 70
k 71
k 72
k 73
k 74
k 75
k 76
k 77
k 78
k 79
k 80
k 81
k 82
k 83
k 84
k 85
k 86
k 87
k 88
k 89
k 90
k 91
k 92
k 93
k 94
k 95
k 96
k 97
k 98
k 99
k 100
k 101
k 102
k 103
k 104
k 105
k 106
k 107
k 108
k 109
k 110
k 111
k 112
k 113
k 114
k 115
k 116
k 117
k 118
k 119
k 120
k 121
k 122
k 123
k 124
k 125
k 126
k 127
k 128
k 129
k 130
k 131
k 132
k 133
k 134
k 135
k 136
k 137
k 138
k 139
k 140
k 141
k 142
k 143
k 144
k 145
k 146
k 147
k 148
k 149
k 150
k 151
k 152
k 153
k 154
k 155
k 156
k 157
k 158
k 159
k 160
k 161
k 162
k 163
k 164
k 165
k 166
k 167
k 168
k 169
k 170
k 171
k 172
k 173
k 174
k 175
k 176
k 177
k 178
k 179
k 180
k 181
k 182
k 183

In [55]:
len(projets_a_ajouter)

0

In [ ]:
print(df_partenaires)

In [ ]:
len(df_partenaires)

In [ ]:
err=[]
for i,row in df_partenaires.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    try:
       r=requests.post('http://185.161.45.213/projects/participations', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           print(i)
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

Modifications

In [ ]:
nbr_page=int(requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANR","participant_id":{"$exists":false}}&projection={"id":1}&max_results=500'+f"&page={1}", headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])

list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANR","participant_id":{"$exists":false}}&projection={"id":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        print("k",k)
        list_ids.append(page['data'][k]['id'])

df_partenaires = df_partenaires[df_partenaires['id'].apply(lambda x: x in list_ids)]

In [ ]:
len(list_ids)

In [ ]:
df_partenaires=df_partenaires[(df_partenaires.participant_id!='x') & (df_partenaires.participant_id!='None')]

In [ ]:
df_partenaires

In [ ]:
err=[]
for i,row in df_partenaires.iterrows() :
    print(i)
    id=row['id']
    url = f"http://185.161.45.213/projects/participations/{id}"
    project=requests.get(url, headers={'Authorization': Authorization}).json()
    head = {"Authorization": Authorization, "If-Match": project['etag'], "Content-Type": "application/json"}

    r = requests.patch(url, json = {"participant_id": row['participant_id']}, headers=head)
    res= r.json()
    if res.get('status')=='ERR':
        err.append(res)
        if res.get('error').get('code')!=422:
            print(err)
            pp.pprint(err)

In [ ]:
pd.Series([x.get('issues').get('id')[25:] for x in err]).drop_duplicates().tolist()

Faire un nouveau fichier avec toutes les structures manquantes

In [ ]:
struct_manq=pd.read_excel('scanr_partenaires_non_identifies.xlsx')

In [ ]:
nom1=struct_manq[struct_manq.New=='IRESP'].drop_duplicates(subset='Nom').Nom

In [ ]:
iresp_actuel=pd.read_excel('./missing_ids_structures/partenaires_non_identifies_IRESP.xlsx')

In [ ]:
nom2=iresp_actuel['Nom_equipe']

In [ ]:
pd.Series(list(nom1)+list(nom2))

In [ ]:
pd.Series(list(nom1)+list(nom2)).drop_duplicates()